# Атрибуты класса и экземпляра

При объявлении класса создается новое пространство имен и используется как локальная область видимости (по аналогии с функциями), внутри которой могут объявляться переменные (атрибуты) и функции, которые называются методами и имеют немного отличный от обычных функций вид объявления. Таким образом все атрибуты можно разделить на несколько видов:
- атрибуты данных:
    - атрибуты класса;
    - атрибуты экземпляры.
- методы (о методах речь пойдет в следующем разделе).

Содержание области видимости класса можно посмотреть с помощью функции ```dir```.

Доступ к любым атрибутам осуществляется с помощью точечной нотации, синтаксис которой является стандартным для многих языков программирования.

```python 
obj.attribute
```

Атрибутами выступают все имена, определенные в пространстве имен класса. Рассмотрим пример простейшего класса ```A```.

## Атрибуты класса

Атрибуты класса объявляются непосредственно внутри пространства имен класса, по аналогии с локальными переменными. В примере ниже класс ```A``` содержит один атрибут класса ```foo```.

Все атрибуты являются ссылками на другие объекты. Здесь атрибут ```foo``` класса ```A``` ссылается на объект целого числа ```6174```.

In [1]:
class A:
    foo = 6174

print(f'{A = }')
print(f'До изменения: {A.foo = }')
A.foo = 0
print(f'После изменения: {A.foo = }')

A = <class '__main__.A'>
До изменения: A.foo = 6174
После изменения: A.foo = 0


Классы позволяют по-настоящему оценить динамическую природу Python. Он позволяет динамически, т.е. в процессе выполнения, создавать **любые** атрибуты.

In [2]:
def get_attrs(obj):
    """Получение все атрибутов кроме 'магических'."""
    return [attr for attr in dir(obj) if not (attr.startswith("__") and attr.endswith("__"))]


print(f'Состав исходной области видимости: {get_attrs(A)}')

# динамически добавляем новый атрибут
A.bar = 42

print(f'Область видимости после добавления атрибута: {get_attrs(A)}')

Состав исходной области видимости: ['foo']
Область видимости после добавления атрибута: ['bar', 'foo']


Однако, обращение к несуществующему атрибуту влечет за собой появление
исключения ```AttributeError```.

In [3]:
class B:
    pass


print(B.attribute)

AttributeError: type object 'B' has no attribute 'attribute'

Экземпляр класса создается с использованием круглых скобок («вызов» класса). Подробнее механизм создания экземпляра класса будет рассмотрен в разделе о "магических" методах. 

Создаваемый экземпляр сразу связывается с именем, стоящим слева от знака равенства.

Экземпляры класса имеют доступ к атрибутам класса. Для этого достаточно использовать точечную нотацию. Изменение таких атрибутов у класса ведет к их изменению в экземплярах, но **обратное не верно**. Далее мы поймем почему. 

In [4]:
a = A()
b = A()

print(f'(1) До изменения: {a.foo = }')
print(f'(2) До изменения: {b.foo = }')

A.foo = 6174

print(f'(1) После изменения: {a.foo = }')
print(f'(2) После изменения: {b.foo = }')

(1) До изменения: a.foo = 0
(2) До изменения: b.foo = 0
(1) После изменения: a.foo = 6174
(2) После изменения: b.foo = 6174


Чтобы убедиться в доступности атрибутом можно воспользоваться функцией ```get_attrs```, реализованной выше.

In [5]:
print(f'Атрибуты класса {A.__name__}: {get_attrs(A)}')

print(f'Атрибуты экземпляра a: {get_attrs(a)}')
print(f'Атрибуты экземпляра b: {get_attrs(b)}')

Атрибуты класса A: ['bar', 'foo']
Атрибуты экземпляра a: ['bar', 'foo']
Атрибуты экземпляра b: ['bar', 'foo']


На самом деле здесь происходит небольшая магия. Непосредственно экземпляры класса не имеют атрибутов класса, т.е. экземпляры ```a``` и ```b``` не имеют атрибутов ```foo``` и ```bar```. Дело в том, что здесь работает механизм поиска атрибутов, о котором мы подробнее поговорим далее. Когда мы запрашиваем атрибут класса у экземпляра, т.е. пишем ```a.foo```, он ищет этот атрибут сначала в экземпляре, не найдя его, он перейдет на уровень выше - на уровень класса и будет искать его там.

Для того, чтобы убедиться в этом проверим на что ссылаются эти атрибуты с помощью оператора ```is```.

In [6]:
# все атрибуты ссылаются на один объект в памяти
print(f'{A.foo is a.foo is b.foo = }')

A.foo is a.foo is b.foo = True


С тем, что атрибутов класса на самом деле нет в экземпляре, связана одна интересная особенность, о которой обязательно нужно помнить. Она может привести к различного рода ошибкам и некорректному поведения программы особенно у новичков. Дело в том, что при создании атрибутов экземпляра с таким же именем как у атрибута класса, первый "загораживает" второй. Это поведение аналогично поведения переменных в областях видимости разного уровня.

В качестве примера попытаемся изменить значение атрибута класса ```foo``` через экземпляр ```b```. Для частоты эксперимента заменим значение атрибута ```b.foo``` на тоже самое значение, которое было ранее у всех остальных атрибутов, а именно ```6174```. Первое, что подумает новичок - ничего не произойдет. Однако, это не так. Подробно разберем происходящее здесь.

Изначально класс ```A``` содержал атрибут класса ```foo```. У экземпляров этого атрибута не было. Они получали его значение через свой класс. 

<img src="image/attr_cls.png">

В ходе выполнения строки ```b.foo = 6174``` интерпретатор сначала выполнил поиск атрибута ```foo``` в экземпляре. Не найдя его, он создал этот атрибут внутри экземпляра со значением ```6174```. Обратите внимание, что при создании атрибута его поиск происходит только **в текущем** объекте. После выполнения этой строки будет существовать два **разных** атрибута ```foo```. Один в экземпляре ```b```, он будет атрибутом экземпляра, а другой в классе ```A```, он является атрибутом класса. Эти два атрибута ссылаются на *разные* объекты, поэтому оператор ```is``` возвращает ```False```. 

Особенности на этом не заканчиваются. Теперь при обращении ```b.foo``` будет возвращаться объект, связанный с этим атрибутом, а не с атрибутом ```A.foo```, т.е. атрибут экземпляра "загородил" собой атрибут класса. Это поведение полностью аналогично обычным переменным из разных областей видимости, например, локальные переменные с тем же именем, что и у глобальной переменной, "загородят" последнюю. Стоит отметить, что теперь атрибут ```b.foo``` никак не будет влиять на ```A.foo```, это **разные** объекты.

In [7]:
print('Атрибуты до изменения:')
print(f'{A.foo = }, {a.foo = }, {b.foo = }')
print('-' * 25)

# Заменяем на такое же значение, по ощущениям ничего не должно измениться.
b.foo = 6174

# Что-то пошло не так. Атрибут b.foo отличается от остальных
print(f'{A.foo is a.foo is b.foo = }')
print(f'{A.foo is b.foo = }')
print(f'{a.foo is b.foo = }')

Атрибуты до изменения:
A.foo = 6174, a.foo = 6174, b.foo = 6174
-------------------------
A.foo is a.foo is b.foo = False
A.foo is b.foo = False
a.foo is b.foo = False


In [8]:
A.foo = 42

print(f'{A.foo = }')
print(f'{a.foo = }')
print(f'{b.foo = }')  # Magic!!!

A.foo = 42
a.foo = 42
b.foo = 6174


До "загороженного" атрибута класса ```foo``` можно добраться из экземпляра. Для этого используется обходной путь. У экземпляра есть "магический" атрибут ```__class__```. Он хранить класс. 

<img src="image/attr_self.png">

In [9]:
print(f'{b.__class__.foo = }')
print(f'{b.__class__.foo is b.foo = }')

b.__class__.foo = 42
b.__class__.foo is b.foo = False


In [10]:
print(f'{a.__class__.foo = }')
print(f'{a.__class__.foo is a.foo = }')

a.foo = 0
print(f'{a.foo = }')
print(f'{a.__class__.foo = }')
print(f'{a.__class__.foo is a.foo = }')

a.__class__.foo = 42
a.__class__.foo is a.foo = True
a.foo = 0
a.__class__.foo = 42
a.__class__.foo is a.foo = False


## Атрибуты экземпляра

Экземпляр класса тоже может содержать атрибуты. Они называются атрибутами экземпляра. Эти атрибуты создаются и инициализируются в специальном методе ```__init__```. Этот метод всегда принимает в качестве первого аргумента ```self```. В качестве этого аргумента передается сам экземпляр. Более подробно работа методов будет рассмотрена в следующем разделе.

Метод ```__init__``` только инициализирует экземпляр, т.е. создает атрибуты и заполняет их определенными значениями. Обратите внимание на то, что метод ```__init__``` не должен ничего возвращать, т.е. в нем нельзя размещать инструкцию ```return```.

In [11]:
class A:
    # метод инициализации экземпляра
    def __init__(self, x, y):
        """
        :param self: экземпляр текущего класса, обязательный аргумент
        :type self: A
        :param x: дополнительное значение для атрибута self.x
        :param y: дополнительное значение для атрибута self.y
        """
        self.x = x
        self.y = y

К атрибутам экземпляра класс не имеет доступа. В этом можно убедиться с помощью функции ```get_attrs```. Область видимости класса ```A``` не содержит этих атрибутов.

In [12]:
print(f'{get_attrs(A)}')

[]


Когда мы объявили метод инициализации экземпляра, его создание несколько измениться. Теперь при вызове класса необходимо передавать дополнительные аргументы ```x``` и ```y```. Обратите внимание, что аргумент ```self``` мы не передаем. Он передается автоматически.

После создания и инициализации экземпляра атрибуты ```x``` и ```y``` появятся в его области видимости. При это они будут различны для разных экземпляров.

In [13]:
a = A(1, 2)
b = A(4, 5)

print(f'{get_attrs(a)}')
print(f'{get_attrs(b)}')

['x', 'y']
['x', 'y']


In [14]:
print(f'{a.x = }, {a.y = }')
print(f'{b.x = }, {b.y = }')

print(f'{a.x is b.x = }, {a.y is b.y = }')

a.x = 1, a.y = 2
b.x = 4, b.y = 5
a.x is b.x = False, a.y is b.y = False


Изменение атрибута у одного из экземпляров никак не отражается на другом.

In [15]:
a.x = 10
print(f'{a.x = }, {a.y = }')
print(f'{b.x = }, {b.y = }')

a.x = 10, a.y = 2
b.x = 4, b.y = 5


Наличие атрибуток класса позволяет хранить одно общее состояние для всех создаваемых объектов. Одним из самых простых применений явялется глобальный счетчик.

In [16]:
class Counter:
    global_counter = 0
    def __init__(self, initial=0):
        self.counter = initial
    
    def increment(self):
        self.counter += 1
        self.__class__.global_counter += 1
    
    def get_local_counter(self):
        return self.counter
    
    def get_global_counter(self):
        return self.__class__.global_counter

In [19]:
x = Counter(5)
x.increment()
y = Counter(42)
y.increment()
y.increment()

In [18]:
print(f'{x.get_local_counter() = }')
print(f'{x.global_counter = }')
print(f'{y.get_local_counter() = }')
print(f'{y.global_counter = }')

6
3
44
3


Также можно считать количество создаваемых объектов, контролировать создание экземпляров и т.д.

# Полезные ссылки